# 3.3 再帰関数とブロック付きメソッド

3.2では、副作用の危険性や参照透過性を保った関数の利点を説明した。本章では副作用を利用しない反復手法や、より抽象的にプログラムを記述する手法を説明する。

## 再帰関数

Rubyにおける反復手法の一つは、2章で説明したループ(*for*, *while*)である。しかし、以下のような例の場合は、変数*res*の値が変更され続けるため、参照透過性を満たしていない。

In [1]:
def factorial(n)
    res = 1
    for i in (1..n)
        res *= i
    end
    res
end

factorial(5)

120

ここで自身の関数を呼び出す**再帰関数**を利用する。再帰関数を考える際には、数学的帰納法の**基底段階**と**帰納段階**を考える事と良い。
*factorialR*(Rは再帰recursiveのR)の場合には、以下のように考える。

* 基底段階: ```n=0```のとき ```n! = 1``` とする
* 帰納段階: ```n!```が分かっているとき、```(n + 1)! = (n + 1) * n!``` とする

このとき 帰納段階では、左辺の(n + 1)! を定義するのに右辺のn!を用いている。
すなわち、階乗を用いて階乗を定義している。
階乗を表す関数を*factrialR*とすると、関数*factialR*(n)は、
```
f(n) = n * factorialR(n - 1)
```
というふうに、自分自身を自分で再帰的(recursive)に呼ぶことで、定義できる。

In [2]:
def factorialR(n)
    if n == 0
        1
    else
        n * factorialR(n - 1)
    end
end

factorialR(5)

120

再帰関数を利用することによって、問題を分割し段階的に考えることができ、また、*factorial*の状態を扱う*res*のような変数が減り、シンプルに記述できる場合がある。それが顕著な例の一つがユークリッド互除法である。


> ユークリッドの互除法（ユークリッドのごじょほう、英: Euclidean Algorithm）は、2 つの自然数の最大公約数を求める手法の一つである。
2 つの自然数 a, b (a ≧ b) について、a の b による剰余を r とすると、 a と b との最大公約数は b と r との最大公約数に等しいという性質が成り立つ。この性質を利用して、 b を r で割った剰余、 除数 r をその剰余で割った剰余、と剰余を求める計算を逐次繰り返すと、剰余が 0 になった時の除数が a と b との最大公約数となる。

引用: wikipedia https://ja.wikipedia.org/wiki/ユークリッドの互除法

これをwhileループを用いた手続き的な手法を用いるとaやbの値を更新する必要があり、理解が難しいコードになる。

In [3]:
def gcd(a, b)
    while(b != 0)
        r = a % b
        a = b
        b = r
    end
    a
end

gcd(3, 15)

3

一方、再帰関数を用いると、

* b == 0のとき、 a が最大公約数
* それ以外のとき、bとaのbによる余剰の最大公約数

と問題を分けて考えられるため、非常にわかりやすく記述することができる。


In [4]:
def gcdR(a, b)
    if b == 0
        a
    else
        gcdR(b, a % b)
    end
end

gcdR(3, 15)

3

## 配列と再帰関数

配列(リスト)のようなデータ構造の場合、以下のように構造を分割できる。このような構造のことを再帰的データ構造と呼ぶ。

* 空のリスト
* 先頭の要素 + 残りのリスト

再帰的なデータ構造の場合は、再帰関数の考えが適用しやすい。配列の長さを求める関数*length*の場合は、以下のように考えることができる。

* 空のとき 0
* 空ではないとき 1 + 残りのリストの長さ

3.2で学んだ多重代入を利用すると先頭の要素と残りのリストを取得することができる。

In [5]:
head, *tail = [1, 2, 3]
p head
p tail

1
[2, 3]


[2, 3]

これらを利用すると、*length*の定義は以下のようになる。

In [6]:
def length(arr)
    if arr.empty?
        0
    else
        head, *tail = arr
        1 + length(tail)
    end
end

length([1, 2, 3])

3

配列のすべての要素を一つずつ見ていくような処理では、全て同じ形で一般化する事ができる。

## ブロック付きメソッドと再帰
Rubyの機能の一つに関数に処理(関数)を渡すことができる**ブロック**と言う仕組みがある。ブロックを引数に持つメソッドを**ブロック付きメソッド**と呼ぶ。

block付きメソッドを呼び出し方法は、波括弧{}を用いた方法とdo~endを用いた方法二種類がある。それぞれ同じ意味だが、1行で済む場合には波括弧、複数行に渡る場合には、do~endを用いた方法と使い分けるスタイルが推奨されている。

```ruby
method() { |x| xを用いた処理 }

method() do |x|
    # xを用いた処理
end
```

このブロックによる処理は関数とみなすことができるので、以下と同じように考えることができる。

```ruby
def block(x)
    # xを用いた処理
end
```

それでは、ブロックを持った関数は、どのように定義されるかを見ていこう。ブロックは、&を付けた変数として引数に渡される。このブロックを呼び出すには、*block.call*メソッドを呼び出す。

In [7]:
def doBlock(x, &block)
    block.call(x)
end

doBlock(5) { |x| puts x }

5


単純にブロックを実行する場合には、*yield*キーワードを利用すると短く記述することができ、引数の*&block*も省略可能である。

In [8]:
def doBlock(x)
    yield(x)
end

doBlock(5) { |x| puts x }

5


しかし、再帰関数や別の関数に*&block*を渡す場合には、*yield*による省略はできるが、明示的に関数に*&block*を渡す必要がある。

In [9]:
def times(t, &block)
    if t <= 0
        nil
    else
        yield
        times(t - 1, &block)
    end
end

times(5) { puts "hello" }

hello
hello
hello
hello
hello


配列の要素毎に関数を実行する*each*関数は、以下のように定義でき、*lenght*関数と同じような再帰の形になる。

In [10]:
def each(arr, &block)
    if arr.empty?
        nil
    else
        head, *tail = arr
        block.call(head)
        each(tail, &block)
    end
end

:each

ブロック付きメソッドを利用することで、ループによるデータの反復方法について考える必要が無く、各要素に対してどのような処理をするかというロジック部分だけに集中することが出来るのが利点である。

In [11]:
each([1, 2, 3]) { |x| puts x }

1
2
3


In [12]:
each([1, 2, 3]) { |x| puts x * x }

1
4
9


## チェックリスト

* 再帰関数によりメリットは何か
* ブロック付きメソッドとは何か
* ブロック付きメソッドの利点は何か